In [ ]:
from deepface import DeepFace
from numpy import dot
from numpy.linalg import norm
import numpy as np
import cv2 

prototxt_path = "deploy.prototxt"
model_path = "res10_300x300_ssd_iter_140000.caffemodel"
face_net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

In [29]:
def detect_face(image):
    h, w = image.shape[:2]
    
    blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size=(300, 300),
                                 mean=(104.0, 177.0, 123.0), swapRB=False, crop=False)
    face_net.setInput(blob)
    detections = face_net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.2:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x1, y1, x2, y2) = box.astype("int")

            padding = 20
            x1, y1, x2, y2 = max(0, x1 - padding), max(0, y1 - padding), min(w, x2 + padding), min(h, y2 + padding)

            face = image[y1:y2, x1:x2]
            return face 

    return None 


In [30]:
def extract_features(image):
    image = cv2.resize(image, (256, 256)) 
    
    try:
        features = DeepFace.represent(image, model_name="Facenet", enforce_detection=False)
        # print("📌 Features Output:", features)
        
        if isinstance(features, list) and len(features) > 0:
            return np.array(features[0]['embedding'])
        else:
            print("⚠️ No embedding found")
            return None
    except Exception as e:
        print(f"⚠️ Error extracting features: {e}")
        return None

In [31]:
import sqlite3

db_path = "face_database.db"

# สร้างตารางเก็บข้อมูล
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        firstname TEXT,
        lastname TEXT,
        embedding BLOB
    )
""")
conn.commit()
conn.close()


In [32]:
def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

def shear_image(image, shear_factor=0.2):
    height, width = image.shape[:2]
    M = np.float32([[1, shear_factor, 0], [shear_factor, 1, 0]])
    return cv2.warpAffine(image, M, (width, height))


In [33]:
def generate_features(detect_image):
    features = []
    transformations = {
        "original": detect_image,
        "flipped": cv2.flip(detect_image, 1),
        "rotated": cv2.warpAffine(detect_image, cv2.getRotationMatrix2D((detect_image.shape[1]//2, detect_image.shape[0]//2), 15, 1), (detect_image.shape[1], detect_image.shape[0])),
        "stretched": cv2.resize(cv2.resize(detect_image, (int(detect_image.shape[1] * 1.05), detect_image.shape[0])), (detect_image.shape[1], detect_image.shape[0])),
        "blurred": cv2.GaussianBlur(detect_image, (5,5), 0),
        "clahe": apply_clahe(detect_image),
        "sheared": shear_image(detect_image, 0.2)
    }
    
    for name, img in transformations.items():
        feature = extract_features(img)
        if feature is not None:
            features.append(feature)
    
    return features


In [34]:
def register_face(firstname, lastname, image_path):
    features = []
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        image = cv2.imread(image_path)
        detect_image = detect_face(image)
        
        if detect_image is None:
            print('ไม่พบใบหน้า')
            return False
        
        features = generate_features(detect_image)

        vectorizer = np.array(features)
        final_vector = np.mean(vectorizer, axis=0) 
        
        print(final_vector)

        cursor.execute("INSERT INTO users (firstname, lastname, embedding) VALUES (?, ?, ?)", 
                           (firstname, lastname, final_vector.tobytes()))

        conn.commit()
        conn.close()
        print("ลงทะเบียนสำเร็จพร้อมการขยายข้อมูล!")
        return True
    except Exception as e:
        print(f"Error: {e}")
        return False


In [ ]:
def login_face(firstname, lastname, image_path):
    features = []
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        image = cv2.imread(image_path)
        detect_image = detect_face(image)
        
        if detect_image is None:
            print('ไม่พบใบหน้า')
            return False
        
        features = generate_features(detect_image)

        vectorizer = np.array(features)
        final_vector = np.mean(vectorizer, axis=0) 
        
        #print(vectorizer)
        cursor.execute("SELECT embedding FROM users WHERE firstname = ? AND lastname = ?", (firstname, lastname))
        user_data = cursor.fetchall()
        
        conn.close()
        
        if not user_data:
            print("ไม่พบข้อมูลผู้ใช้ในระบบ")
            return False
        
        # ดึง embedding จากฐานข้อมูล
        db_embedding = user_data[0][0]  # ดึง embedding ของผู้ใช้คนแรก
        db_embedding_np = np.frombuffer(db_embedding, dtype=np.float64)

        cosine_similarity = dot(final_vector, db_embedding_np) / (norm(final_vector) * norm(db_embedding_np))
        print("Cosine Similarity:", cosine_similarity)

        # ใช้ cosine similarity เพื่อตัดสินใจ
        if cosine_similarity > 0.6:  # กำหนด threshold ความคล้ายคลึง
            print(f"เข้าสู่ระบบสำเร็จ! ยินดีต้อนรับ {firstname} {lastname}")
            return True
        else:
            print("ไม่สามารถเข้าสู่ระบบได้: ใบหน้าไม่ตรงกับข้อมูลในระบบ")
            return False

    except Exception as e:
        print(f"Error: {e}")
        return False


In [40]:
if __name__ == "__main__":
    # register_face('Shuhua', 'Shuhua', 'Shuhua01.png')
    login_face('Shuhua', 'Shuhua', 'Minnie02.png')

Cosine Similarity: 0.4968624418820418
ไม่สามารถเข้าสู่ระบบได้: ใบหน้าไม่ตรงกับข้อมูลในระบบ
